# <center> Inteligencia Artificial y Aprendizaje </center> 
## <center> Práctica 3 </center> 
#### <center> *Ana San Román Gaitero y María González García* </center> 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import seaborn as sbn
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,recall_score, precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import learning_curve, GridSearchCV,validation_curve
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from mlxtend.plotting import plot_learning_curves
from sklearn.neural_network import MLPClassifier
from sklearn.utils import resample
from sklearn.tree import export_graphviz
#from sklearn.externals.six import StringIO  
from six import StringIO
from IPython.display import Image  
import pydotplus

In [2]:
# Cargar la base de datos
db = pd.read_csv('pima_indian_diabetes.csv')

## Preprocesado

In [3]:
# Tratamiento Missing values 

db['Glucose'] = db['Glucose'].replace(0, np.nan)
db['BloodPressure'] = db['BloodPressure'].replace(0, np.nan)
db['SkinThickness'] = db['SkinThickness'].replace(0, np.nan)
db['Insulin'] = db['Insulin'].replace(0, np.nan)
db['BMI'] = db['BMI'].replace(0, np.nan)

In [4]:
# Detección de outliers
def outliers (datos):
    count=0
    lista=[]
    for i in db[datos]:
        if not (db[datos].mean()-2.5*db[datos].std())<=i<=(db[datos].mean()+2.5*db[datos].std()):
            if not np.isnan(i):
                count=count+1
                lista.append(i)
   
    return lista

In [5]:
for i in db:
    db[[i]]=db[[i]].replace(outliers(i),np.nan)

In [6]:
print(db.isnull().sum())

Pregnancies                  14
Glucose                       7
BloodPressure                52
SkinThickness               231
Insulin                     391
BMI                          20
DiabetesPedigreeFunction     20
Age                          21
Outcome                       0
dtype: int64


In [7]:
db = db.fillna(np.mean(db))

In [8]:
# Balanceo de los datos 

# Separación de la clase mayoritaria (no diabéticos) de la clase minoritaria (diabéticos)

db_majority = db[db.Outcome==0]
db_minority = db[db.Outcome==1]
 
# Downsample majority class
db_majority_downsampled = resample(db_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=268,     # to match minority class
                                 random_state=2) # reproducible results
 
# Combine minority class with downsampled majority class
db = pd.concat([db_majority_downsampled, db_minority])
 
# Display new class counts
db.Outcome.value_counts()

1    268
0    268
Name: Outcome, dtype: int64

In [9]:
# Obtenemos X e Y las separamos

X = db.drop(['Outcome'], axis = 1)     # elimina la variable Outcome
Y = db['Outcome']

print(X.shape)
print(Y.shape)

(536, 8)
(536,)


In [10]:
# Separamos el conjunto de datos en los conjuntos de train y test

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=2)

In [17]:
X_train['Glucose']

2      183.000000
305    120.000000
630    114.000000
319    194.000000
522    114.000000
          ...    
577    118.000000
72     126.000000
661    121.687254
749    162.000000
7      115.000000
Name: Glucose, Length: 375, dtype: float64

In [18]:
Y_train

2      1
305    0
630    1
319    1
522    0
      ..
577    1
72     1
661    1
749    1
7      0
Name: Outcome, Length: 375, dtype: int64

In [ ]:
# Normalizamos los datos después de haberlos separado en Train y Test

X_norm = preprocessing.scale(X_train)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

print(X_train_norm.shape)
print(X_test_norm.shape)

## K-NN
**Considere la normalización de características del espacio original de entrada al modelo. Justifique el tipo de normalización considerada y explique cómo lo realiza sobre cada subconjunto (entrenamiento, validación y test).**

Se considera la normalización estandar, ya que transforma los datos a una distribución más sencilla y más conocida. Dejando a todas las características con la misma media 0 y varianza 1. 

La función *StandardScaler* calcula la media y la desviación a partir de los datos del conjunto de training. Mediante esos estadísticos se realiza la normalización tanto en el conjunto de entrenamiento como en el de test. Además, la normalización se aplica de forma independiente para cada característica. Como el conjunto de validación procede del conjunto de train, al utilizar éste posteriormente en cross validation, ya estará normalizado.

**Indique si las prestaciones obtenidas en el conjunto de test cambian tras normalizar las variables.**

- Sin normalizar

In [ ]:
k_values=range(1,31,2)
cv_k_scores=[]

for k in k_values:
    kNN=KNeighborsClassifier(n_neighbors=k) #clasificador knn
    scores= cross_val_score(kNN, X_train, Y_train, cv=3, scoring='accuracy') 
    #calcula el accuracy con el clasificador knn en train y 3 folds
    cv_k_scores.append(scores.mean()) #almacena la media de los scores
    
plt.plot(k_values, cv_k_scores)
plt.xlabel('k')
plt.ylabel('CV accuracy')
plt.show()

# Seleccionamos el máximo valor porque estamso considerando entropía
print("El valor de k más adecuado es",np.array(k_values)[cv_k_scores.index(np.array(cv_k_scores).max())])
print("Accuracy:", max(cv_k_scores))

In [ ]:
KNN = KNeighborsClassifier(n_neighbors = 17)
KNN.fit(X_train, Y_train)

y_pred = KNN.predict(X_test)

print("Accuracy: ",accuracy_score(Y_test, y_pred))
print("Sensibility: ",recall_score(Y_test, y_pred))
print("Confusión matrix: ")
print(confusion_matrix(Y_test, y_pred))
print("Specificity: ",confusion_matrix(Y_test, y_pred)[0][0]/(confusion_matrix(Y_test, y_pred)[0][0]+confusion_matrix(Y_test, y_pred)[0][1]))
print("Precision:", precision_score(Y_test, y_pred))

- Normalizando los datos

In [ ]:
k_values=range(1,31,2)
cv_k_scores=[]

for k in k_values:
    kNN=KNeighborsClassifier(n_neighbors=k) #clasificador knn
    scores= cross_val_score(kNN, X_train_norm, Y_train, cv=3, scoring='accuracy') 
    #calcula el accuracy con el clasificador knn en train y 3 folds
    cv_k_scores.append(scores.mean()) #almacena la media de los scores
    
plt.plot(k_values, cv_k_scores)
plt.xlabel('k')
plt.ylabel('CV accuracy')
plt.show()

# Seleccionamos el máximo valor porque estamso considerando entropía
print("El valor de k más adecuado es",np.array(k_values)[cv_k_scores.index(np.array(cv_k_scores).max())])
print("Accuracy:", max(cv_k_scores))

In [ ]:
kNN_norm = KNeighborsClassifier(n_neighbors = 15)
kNN_norm.fit(X_train_norm, Y_train)

y_pred1 = kNN_norm.predict(X_test_norm)

print('Accuracy:', accuracy_score(Y_test, y_pred1))
print('Sensibility:',recall_score(Y_test, y_pred1))
print("Confusión matrix: ")
print(confusion_matrix(Y_test, y_pred1))
print('Specificity:',confusion_matrix(Y_test, y_pred1)[0][0]/(confusion_matrix(Y_test, y_pred1)[0][0]+confusion_matrix(Y_test, y_pred1)[0][1]))
print("Precision:", precision_score(Y_test, y_pred1))

In [ ]:
Tabla = pd.DataFrame({ "Prestaciones":["Accuracy","Sensibility", 'Specificity','Precision'],
                      "kNN sin normalizar k=17" : ["0.72","0.80", '0.637','0.69'],
                      "kNN normalizado k=15" : ["0.733","0.78", '0.6875','0.72']})
Tabla


Normalizando y sin normalizar las variables se obtienen distintos valores óptimos de k. Cuando k=17 (sin normalizar) se obtiene un accuracy, precisión y especificidad ligeramente menor y por el contrario, una sensibilidad mayor. Se podría entender que es mejor no normalizar los datos para detectar mejor a los casos diabéticos, pero al calcular distancias euclídeas es indispensable la normalización, ya que sin ella los resultados obtenidos no se ajustarían a la realidad, pues se encuentran en diferentes rangos dinámicos. Es por ello, que con un k=15 la sensibilidad, aunque sea ligeramente menor, es mucho más fiable (hay una precisión mayor). Es decir, tiene una mayor seguridad de detectar correctamente a los diabéticos.

En conclusión, se obtienen mejores resultados en el conjunto de test tras haber normalizado las variables.

**Explique razonadamente la relación entre la capacidad de generalización y el valor de k. Incluya las curvas de obtenidas para una medida de evaluación considerada tanto para el conjunto de train como para el de validación en función del parámetro k. Coméntelas brevemente.**

El valor de k influye significativamente en la capacidad de generalización del algoritmo. Un valor de k pequeño producirá fronteras altamente complejas y flexibles, sobreajustando el clasificador. Un valor de k grande puede dar lugar a fronteras demasiado suaves que no generalicen adecuadamente el modelo. Por lo tanto, es muy importante la correcta elección del valor de k.

In [ ]:
train_score, val_score = validation_curve(KNeighborsClassifier(), X_train_norm, Y_train,
                                          'n_neighbors', k_values, cv=3)

plt.plot(k_values, np.mean(train_score, 1), color='blue', label='Training accuracy')
plt.plot(k_values, np.mean(val_score, 1), color='red', label='Validation accuracy')
plt.legend(loc='best')
plt.ylim(0, 1.2)
plt.xlabel('k')
plt.ylabel('Accuracy');

La curva de entrenamiento sigue un comportamiento descendente hasta que llega a un punto que se estabiliza. Con valores pequeños de k se consigue un mayor accuracy, pues el modelo se esta diseñando en base a los datos de entrenamiento y las fronteras se ajustan perfectamente a dichos datos. Sin embargo, cada vez que k se va haciendo más grande, el accuracy se va haciendo más pequeño porque las fronteras se van volviendo más suaves, no ajustandose tan bien a los datos.

Por otro lado, la curva de validación sigue un comportamiento contrario a la curva de entrenamiento. Al calcular el accuracy del conjunto de validación en un modelo demasiado flexible, con un valor de k pequeño, no se clasificaría correctamente ya que dicho modelo ha memorizado los datos del conjunto de entrenamiento. A medida que k aumenta, el accuracy va mejorando ya que el modelo ha conseguido fronteras que generalizan mejor y no memorizan los datos de entrenamiento, hasta que el valor de k es demasiado grande donde empieza a subajustar y no clasifica correctamente.

## Árboles de decisión

**Indique si las prestaciones obtenidas en el conjunto de test cambian tras normalizar las variables.**

In [ ]:
# GridSearchCV
param_grid = { 'criterion':['gini','entropy']}

# Creamos un árbol de clasificación
dtree_model=tree.DecisionTreeClassifier(random_state=1)

# Usamos gridsearch para evluar los parámetros
dtree_model = GridSearchCV(dtree_model, param_grid, cv=3)

# Entrenamos con los datos de train
dtree_model=dtree_model.fit(X_train, Y_train)
print(dtree_model.best_params_)

- Sin normalizar

In [ ]:
max_depth = np.arange(1, 21)
cv_depth_scores=[]
for k in max_depth:
    dtree = tree.DecisionTreeClassifier(max_depth = k, random_state=1) #clasificador knn
    scores= cross_val_score(dtree, X_train_norm, Y_train, cv=3, scoring='accuracy') 
    #calcula el accuracy con el clasificador knn en train y 3 folds
    cv_depth_scores.append(scores.mean()) #almacena la media de los scores
    
plt.plot(max_depth, cv_depth_scores)
plt.xlabel('max_depth')
plt.ylabel('CV accuracy')
plt.show()

# Seleccionamos el máximo valor porque estamso considerando entropía
print("El valor de profundidad más adecuado es ",np.array(max_depth)[cv_depth_scores.index(np.array(cv_depth_scores).max())])
print("El Accuracy es de", max(cv_depth_scores))

In [ ]:
# Entrenamos el modelo
my_model = tree.DecisionTreeClassifier(random_state=1, max_depth = 4)
my_tree=my_model.fit(X_train, Y_train)

#Predeccimos usando X_test
y_predicted = my_tree.predict(X_test)

# Resultados
print('Accuracy:',accuracy_score(Y_test, y_predicted))
print('Sensibility:',recall_score(Y_test, y_predicted))
print("Confusión matrix: ")
print(confusion_matrix(Y_test, y_predicted)) 
print('Specificity:',confusion_matrix(Y_test, y_predicted)[0][0]/(confusion_matrix(Y_test, y_predicted)[0][0]+confusion_matrix(Y_test, y_predicted)[0][1]))

- Normalizando los datos

In [ ]:
cv_depthnorm_scores=[]
for k in max_depth:
    dtree = tree.DecisionTreeClassifier(random_state=1, max_depth = k) #clasificador knn
    scores= cross_val_score(dtree, X_train_norm, Y_train, cv=3, scoring='accuracy') 
    #calcula el accuracy con el clasificador knn en train y 3 folds
    cv_depthnorm_scores.append(scores.mean()) #almacena la media de los scores
    
plt.plot(max_depth, cv_depthnorm_scores)
plt.xlabel('max_depth')
plt.ylabel('CV accuracy')
plt.show()

# Seleccionamos el máximo valor porque estamso considerando entropía
print("El valor de profundidad más adecuado es ",np.array(max_depth)[cv_depthnorm_scores.index(np.array(cv_depthnorm_scores).max())])
print("El Accuracy es de", max(cv_depthnorm_scores))

In [ ]:
# Entrenamos el modelo
my_model_norm = tree.DecisionTreeClassifier(random_state=1, max_depth = 4)
my_tree_norm=my_model_norm.fit(X_train_norm, Y_train)

#Predeccimos usando X_test
y_predicted = my_tree_norm.predict(X_test_norm)

# Resultados
print('Accuracy:',accuracy_score(Y_test, y_predicted))
print('Sensibility:',recall_score(Y_test, y_predicted))
print("Confusión matrix: ")
print(confusion_matrix(Y_test, y_predicted))
print('Specificity:',confusion_matrix(Y_test, y_predicted)[0][0]/(confusion_matrix(Y_test, y_predicted)[0][0]+confusion_matrix(Y_test, y_predicted)[0][1]))

In [ ]:
Tabla1=pd.DataFrame({ "Prestaciones":["Accuracy","Sensibility", 'Specificity'],
                      "Árbol sin normalizar" : ["0.73","0.73", '0.74'],
                      "Árbol normalizado" : ["0.73","0.73", '0.74']})
Tabla1

Se observa que las prestaciones en el conjunto de test tras normalizar las variables no han cambiado, como era de esperar. Esto es debido a que, de forma recursiva, se va a dividir el espacio en regiones, escogiendo cada vez una variable distinta. Por lo que, cada decisión es independiente a la anterior y por tanto, da igual tener variables en diferentes rangos dinámicos.

**En términos de subajuste y sobreajuste, explique cuál de ellos es más probable que ocurra si el máximo número de casos por nodo del árbol es muy pequeño. Probarlo.**


Primero se calcula el valor óptimo para el modelo.

In [ ]:
min_samples_split = np.arange(2,256)
cv_nodo_scores=[]
for k in min_samples_split:
    dtree = tree.DecisionTreeClassifier(random_state=1, min_samples_split=k) #clasificador knn
    scores= cross_val_score(dtree, X_train_norm, Y_train, cv=3, scoring='accuracy') 
    #calcula el accuracy con el clasificador knn en train y 3 folds
    cv_nodo_scores.append(scores.mean()) #almacena la media de los scores
    
plt.plot(min_samples_split, cv_nodo_scores)
plt.xlabel('min_samples_split')
plt.ylabel('CV accuracy')
plt.show()

# Seleccionamos el máximo valor porque estamso considerando entropía
print("El valor de casos para dividir un nodo más adecuado es ",np.array(min_samples_split)[cv_nodo_scores.index(np.array(cv_nodo_scores).max())])
print("El Accuracy es de", max(cv_nodo_scores))

En gráfica se observa como los valores más altos previenen que el modelo “memorice”. Pero cuando supera la cantidad de datos de los dos fold que utiliza para diseñar el modelo, el accuracy disminuye drásticamente, ya que el árbol no puede separar el nodo raiz.

- Min número casos nodo pequeño vs min número óptimo

In [ ]:
# Entrenamos el modelo con el valor óptimo
my_model_norm = tree.DecisionTreeClassifier(random_state=1,min_samples_split=143)
my_tree_norm=my_model_norm.fit(X_train_norm, Y_train)

#Predeccimos usando X_test
y_predicted = my_tree_norm.predict(X_test_norm)

# Prestaciones en el conjunto de test
print('Accuracy:',accuracy_score(Y_test, y_predicted))
print('Sensibility:',recall_score(Y_test, y_predicted))
print("Confusión matrix: ")
print(confusion_matrix(Y_test, y_predicted))
print('Specificity:',confusion_matrix(Y_test, y_predicted)[0][0]/(confusion_matrix(Y_test, y_predicted)[0][0]+confusion_matrix(Y_test, y_predicted)[0][1]))

In [ ]:
# Entrenamos el modelo con un valor pequeño de casos por nodo
my_model_norm = tree.DecisionTreeClassifier(random_state=1,min_samples_split=2)
my_tree_norm=my_model_norm.fit(X_train_norm, Y_train)

#Predeccimos usando X_test
y_predicted = my_tree_norm.predict(X_test_norm)

# Resultados
print('Accuracy:',accuracy_score(Y_test, y_predicted))
print('Sensibility:',recall_score(Y_test, y_predicted))
print("Confusión matrix: ")
print(confusion_matrix(Y_test, y_predicted))
print('Specificity:',confusion_matrix(Y_test, y_predicted)[0][0]/(confusion_matrix(Y_test, y_predicted)[0][0]+confusion_matrix(Y_test, y_predicted)[0][1]))

In [ ]:
Tabla1=pd.DataFrame({ "Prestaciones":["Accuracy","Sensibility", 'Specificity'], ############################
                      "min número casos nodo=143" : ["0.71","0.72", '0.7125'],
                      "min número casos nodo=2" : ["0.65","0.68", '0.6125']})
Tabla1

Al elegir un valor de *min_samples_split* muy pequeño (en este caso 2), el árbol sigue partiendo una región hasta que el número de casos que contenga sea igual a 2. Esto provoca que el árbol cree relaciones muy especificas y fronteras muy flexibles, obligando a que el algoritmo memorice en vez de aprender. Se concluye que se obtienen peores resultados cuando el mìnimo numero de casos para dividir un nodo es muy pequeño.

Sin embargo, con un valor de *min_samples_split* = 143, elegido mediante cross validation, consigue generalizar mejor. Es por ello, que se obtienen mejores resultados en el conjunto de test.

- Min número casos en el nodo terminal

In [ ]:
min_samples_leaf = range(1, 126)
cv_leaf_scores=[]
for k in min_samples_leaf:
    dtree = tree.DecisionTreeClassifier(random_state=1,min_samples_leaf=k)
    scores= cross_val_score(dtree, X_train_norm, Y_train, cv=3, scoring='accuracy') 
    cv_leaf_scores.append(scores.mean()) #almacena la media de los scores
    
plt.plot(min_samples_leaf, cv_leaf_scores)
plt.xlabel('min_samples_leaf')
plt.ylabel('CV accuracy')
plt.show()

# Seleccionamos el máximo valor porque estamso considerando entropía
print("El valor de casos en el nodo terminal más adecuado es ",np.array(min_samples_leaf)[cv_leaf_scores.index(np.array(cv_leaf_scores).max())])
print("El Accuracy es de", max(cv_leaf_scores))

En el gráfico se puede observar que valores grandes de *min_samples_leaf* predicen mejor que los valores pequeños. Pero cuando  la cantidad de datos es mayor a la que hay en cada fold (125) el accuracy disminuye drásticamente, ya que el árbol no puede tener un nodo terminal con un número menor a 125. Por ejemplo: si se parte de 250 muestras en el nodo raíz y se tiene un *min_samples_leaf*=126, el árbol se dividiría en dos nodos terminales, teniendo uno de ellos un número de casos terminales que no alcanzaría el mínimo, lo cual no podría dividirse.

**Indique cuáles de los parámetros comentados en teoría ha analizado para conseguir que el modelo diseñado tenga capacidad de generalización.**

Para conseguir que el modelo tenga capacidad de generalización se han analizado el mínimo número de muestras para dividir un nodo, el mínimo número de muestras para un nodo terminal y la máxima profundidad del árbol.

In [ ]:
my_model_norm = tree.DecisionTreeClassifier(random_state=1,max_depth=4,min_samples_split=143,min_samples_leaf=123)
my_tree_norm=my_model_norm.fit(X_train_norm, Y_train)
y_predicted=my_tree_norm.predict(X_test_norm)
print('Accuracy:',accuracy_score(Y_test, y_predicted))
print('Sensibility:',recall_score(Y_test, y_predicted))
print("Confusión matrix: ")
print(confusion_matrix(Y_test, y_predicted))
print('Specificity:',confusion_matrix(Y_test, y_predicted)[0][0]/(confusion_matrix(Y_test, y_predicted)[0][0]+confusion_matrix(Y_test, y_predicted)[0][1]))
print("Precision:", precision_score(Y_test, y_predicted))

Según los parámetros obtenidos con cross validation se obtiene el siguiente árbol.

In [ ]:
dot_data = StringIO()
export_graphviz(my_tree_norm, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = db.columns[0:-1],class_names=['Diabetic','Non Diabetic'])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('diabetes.png')
Image(graph.create_png())

Se obtiene un árbol con profundidad 1, el mínimo número de casos en el nodo terminal igual a 123 y el mínimo número de casos para dividir un nodo igual a 143; que según las prestaciones obtenidas en validación y en test da buenos resultados, accuracy aproximadamente de 0'7. Sin embargo, teóricamente dichos valores de los parámetros estarían subajustando. Se aconsejaría realizar el proceso varias veces para diferentes particiones y balanceos de datos.

**Incluya las curvas de obtenidas para una medida de evaluación considerada tanto para el conjunto de train como para el de validación en función de cada parámetro. Coméntelas brevemente.**

    * Profundidad del arbol

In [ ]:
max_depth=np.arange(2,21)

train_score, val_score = validation_curve(tree.DecisionTreeClassifier(criterion='gini'), X_train_norm, Y_train,
                                          'max_depth', max_depth, cv=3)

plt.plot(max_depth, np.mean(train_score, 1), color='blue', label='Training accuracy')
plt.plot(max_depth, np.mean(val_score, 1), color='red', label='Validation accuracy')
plt.legend(loc='best')
plt.ylim(0.5, 1.1)
plt.xlabel('Depth')
plt.ylabel('Accuracy');

La curva de entrenamiento sigue un comportamiento ascendente hasta que llega a un punto que se estabiliza. Con valores grandes de *max_depth* se consigue un mayor accuracy, pues el clasificador se esta diseñando en base a los datos de entrenamiento y esta creando relaciones específicas a dichos datos. Sin embargo, para valores pequeños, el accuracy es menor ya que el árbol no divide los nodos necesarios para definir a sus datos, por la limitación de su profundidad.

Por otro lado, la curva de validación sigue un comportamiento contrario a la curva de entrenamiento. Teóricamente, esta curva debería tener una forma de U invertida, donde valores pequeños de profundidad subajustarían (accuracy bajo) y valores grandes de profundidad sobreajustarían obteniendo un accuracy bajo. Sin embargo, esto es muy difícil de ver en la práctica, por ello obtenemos una curva de validación que comienza con un accuracy bajo hasta 4, y luego sigue disminuyendo hasta que se estabiliza.

    * Min número casos para dividir un nodo

In [ ]:
min_samples_split = np.arange(2,150)

train_score, val_score = validation_curve(tree.DecisionTreeClassifier(random_state=1), X_train_norm, Y_train,
                                          'min_samples_split', min_samples_split, cv=3)

plt.plot(min_samples_split, np.mean(train_score, 1), color='blue', label='Training accuracy')
plt.plot(min_samples_split, np.mean(val_score, 1), color='red', label='Validation accuracy')
plt.legend(loc='best')
plt.ylim(0.5, 1)
plt.xlabel('Min number samples')
plt.ylabel('Accuracy');

La curva de entrenamiento sigue un comportamiento descendente hasta que llega a un punto que se estabiliza. Con valores pequeños de *min_samples_split* se consigue un mayor accuracy, ya que se crea un árbol más específico y se ha creado a partir de los datos de entrenamiento. Sin embargo, para valores grandes, el accuracy es menor ya que el árbol no divide los nodos necesarios para definir a sus datos.

Por otro lado, la curva de validación sigue un comportamiento ascendente hasta un determinado valor óptimo, a partir del cual desciende y se estabiliza. En esta curva se asemeja mejor al comportamiento teórico en comparación con la curva de validación de profundidad. Para valores grandes de *min_samples_split* subajusta y para valores pequeños sobreajusta.

    * Min número casos para un nodo terminal

In [ ]:
min_samples_leaf = np.arange(1, 50)

train_score, val_score = validation_curve(tree.DecisionTreeClassifier(random_state=1), X_train_norm, Y_train,
                                          'min_samples_leaf', min_samples_leaf, cv=3)

plt.plot(min_samples_leaf, np.mean(train_score, 1), color='blue', label='Training accuracy')
plt.plot(min_samples_leaf, np.mean(val_score, 1), color='red', label='Validation accuracy')
plt.legend(loc='best')
plt.ylim(0.5, 1)
plt.xlabel('min_samples_leaf')
plt.ylabel('Accuracy');

La curva de entrenamiento de *min_samples_leaf* sigue el mismo comportamiento que la de *min_samples_split*, teniendo las mismas razones que ésta al igual que la de validación. Para valores grandes de *min_samples_leaf* subajusta y para valores pequeños sobrejusta.

## Perceptrón Multicapa

**Haciendo uso de las características del conjunto de observaciones en el espacio original, diseñe un Perceptrón Multicapa (MLP) con una única capa oculta, inicialización aleatoria de los pesos de la red neuronal, aprendizaje en modo mini-batch (por mini-lotes), algoritmo de entrenamiento back- propagation y detención del aprendizaje haciendo uso de la técnica “early stopping”.
Justifique razonadamente:**

- El número de neuronas de entrada al MLP:

El número de neuronas de entrada al MLP será igual al número de carcaterísticas que tengamos en la base de datos (Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age) en este caso, 8 neuronas de entrada. 

- El número de neuronas de salida del MLP.

Al tener una variable de salida binaria (Outcome), el número de neuronas de salida del MLP será 1

- Las funciones de activación consideradas.

In [ ]:
cv_act=["logistic","tanh","relu"]
cv_act_scores=[]
for k in cv_act:
    MLP = MLPClassifier(random_state=1,max_iter=5000, activation=k)
    scores= cross_val_score(MLP, X_train_norm, Y_train, cv=3, scoring='accuracy') 
    print('Score',scores)
    cv_act_scores.append(scores.mean()) #almacena la media de los scores
    

plt.plot(cv_act, cv_act_scores)
plt.xlabel('Función de activación')
plt.ylabel('CV accuracy')
plt.show()

print("La función de acivación más adecuada es:",np.array(cv_act)[cv_act_scores.index(np.array(cv_act_scores).max())])
print("Accuracy:", max(cv_act_scores))


Mediante cross validation se observa que la función de activación que mejor accuracy presenta en el conjunto de validación es *logistic*. Por tanto, se elige para crear el clasificador MLP.

In [ ]:
#Prestaciones en el conjunto de test
MLP = MLPClassifier(random_state=1, activation="logistic",max_iter=5000) 

MLP_model=MLP.fit(X_train_norm, Y_train)
y_pred=MLP_model.predict(X_test_norm)
print('Accuracy:', accuracy_score(Y_test, y_pred))
print('Sensibility',recall_score(Y_test, y_pred))

- El valor de la tasa de aprendizaje utilizada, así como variación (o no) de la misma durante el aprendizaje.

El tipo de tasa de aprendizaje o *learning_rate* solo es aplicable para el descenso del gradiente. Por ello, para calcular estos parametros de tasa de aprendizaje, primero es necesario ver que solver es el más indicado en nuestro modelo MLP.

In [ ]:
solver=["lbfgs", "sgd", "adam"]
cv_sol_scores=[]

for k in solver:
    MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver=k)
    scores= cross_val_score(MLP, X_train_norm, Y_train, cv=3, scoring='accuracy') 
    #calcula el accuracy con el clasificador MLP en train y 3 folds
    print('Score',scores)
    cv_sol_scores.append(scores.mean()) #almacena la media de los scores
    

    
plt.plot(solver, cv_sol_scores)
plt.xlabel('Solver')
plt.ylabel('CV accuracy')
plt.show()


print("El solver más adecuado es:",np.array(solver)[cv_sol_scores.index(np.array(cv_sol_scores).max())])
print("Accuracy:", max(cv_sol_scores))

Mediante cross validation se observa que los solver que mejor accuracy presentan en el conjunto de validación son *sgd* y *adam*. Por tanto, se elige *sgd* para crear el clasificador MLP debido a que *adam* principalmente se utiliza para grandes bases de datos.

In [ ]:
MLP = MLPClassifier(random_state=1, activation="logistic",max_iter=5000, solver="sgd") 

MLP_model=MLP.fit(X_train_norm, Y_train)
y_pred=MLP_model.predict(X_test_norm)
print('Accuracy:', accuracy_score(Y_test, y_pred))
print('Sensibility',recall_score(Y_test, y_pred))

Como el solver es sgd, debemos calcular el learning rate y learning rate init

    * Learning rate 

In [ ]:
learning_rate=["constant", "adaptive", "invscaling"]
cv_rate_scores=[]

for k in learning_rate:
    MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver='sgd',learning_rate=k)
    scores= cross_val_score(MLP, X_train_norm, Y_train, cv=3, scoring='accuracy') 
    #calcula el accuracy con el clasificador MLP en train y 3 folds
    print('Score',scores)
    cv_rate_scores.append(scores.mean()) #almacena la media de los scores
    
plt.plot(learning_rate, cv_rate_scores)
plt.xlabel('learning_rate')
plt.ylabel('CV accuracy')
plt.show()


print("El learning_rate más adecuado es:",np.array(learning_rate)[cv_rate_scores.index(np.array(cv_rate_scores).max())])
print("Accuracy:", max(cv_rate_scores))

En este caso, la tasa de aprendizaje no variará a medida que desciende el gradiente. Normalmente, en la mayoria de los casos, es mejor una tasa de aprendizaje adaptativa, es decir, que vaya muy rápido al principio y cuando se acerque al mínimo global disminuya para no saltárselo.

In [ ]:
MLP = MLPClassifier(random_state=1, activation="logistic",max_iter=5000, solver="sgd", learning_rate='constant') 

MLP_model=MLP.fit(X_train_norm, Y_train)
y_pred=MLP_model.predict(X_test_norm)
print('Accuracy:', accuracy_score(Y_test, y_pred))
print('Sensibility',recall_score(Y_test, y_pred))

    * Learning rate init

In [ ]:
#Calculo del parámetro learning_rate_init sobre el conjunto de validación 

learning_rate_init = np.linspace(0.0009,0.1)
cv_learning_scores=[]

for k in learning_rate_init:
    
    MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver="sgd",learning_rate='constant',learning_rate_init=k )
    scores= cross_val_score(MLP, X_train_norm, Y_train, cv=3, scoring='accuracy') 
    #calcula el accuracy con el clasificador MLP en train y 3 folds
    #print('Score',scores)
    cv_learning_scores.append(scores.mean()) #almacena la media de los scores
    

    
plt.plot(learning_rate_init, cv_learning_scores)
plt.xlabel('learning_rate_init')
plt.ylabel('CV accuracy')
plt.show()


print("El tasa de aprendizaje más adecuado es:",np.array(learning_rate_init)[cv_learning_scores.index(np.array(cv_learning_scores).max())])
print("Accuracy:", max(cv_learning_scores)) 

Como se puede observar en la gráfica, con valores pequeños se consigue llegar a un mayor accuracy ya que va poco a poco sin saltarse el mínimo global, sin embargo, tiene un mayor coste computacional en cuanto al tiempo. Con valores grandes, en torno a 0.06, el accuracy comienza a disminuir porque la tasa de aprendizaje se salta el mínimo global y llega a un mínimo local de la función de coste.

In [ ]:
#Evaluación de las prestaciones en el conjunto de test

MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver="sgd",learning_rate='constant',learning_rate_init= 0.0029224489795918367) 

MLP_model=MLP.fit(X_train_norm, Y_train)
y_pred=MLP_model.predict(X_test_norm)
print('Accuracy:', accuracy_score(Y_test, y_pred))
print('Sensibility',recall_score(Y_test, y_pred))

- Función de coste a optimizar.

La función de coste elegida es Cross Entropy ya que contamos con un problema de clasificación binaria.

- Los valores numéricos que representan el número de neuronas consideradas en la capa oculta. Justifique la consideración de cada uno de los valores indicados (al menos, tres valores:n_hidden_1, n_hidden_2 y n_hidden_3).

El número de neuronas en la capa ocula es un parámetro a elegir por medio de cross validation. 

In [ ]:
#Calculo del parámetro de número de neuronas en la capa oculta sobre el conjunto de validación 
neuronas = []
for i in range(1,80):
    neuronas.append((i,))

cv_neu_scores=[]

for k in neuronas:
    
    MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver="sgd",learning_rate='constant',learning_rate_init= 0.0029224489795918367,hidden_layer_sizes=k)
    scores= cross_val_score(MLP, X_train_norm, Y_train, cv=3, scoring='accuracy') 
    #calcula el accuracy con el clasificador MLP en train y 3 folds
    #print('Score',scores)
    cv_neu_scores.append(scores.mean()) #almacena la media de los scores
       
plt.plot(neuronas, cv_neu_scores)
plt.xlabel('neuronas')
plt.ylabel('CV accuracy')
plt.show()


print("El numero de neuronas más adecuado es:",np.array(neuronas)[cv_neu_scores.index(np.array(cv_neu_scores).max())])
print("Accuracy:", max(cv_neu_scores))

El número de pesos se incrementará cuantas más neuronas haya en la capa oculta generando un modelo más complejo. Esto produce que el clasificador sea menos lineal y por lo tanto, cueste más llegar a una solución óptima (mínimo global). En la gráfica, cuanto mayor se va haciendo el número de neuronas, los picos ya no llegan a los valores de accuracy que se obtenían con los valores pequeños de neuronas. Es por ello, que se obtiene un valor de 29 neuronas en la capa oculta. 

In [ ]:
#Evaluación de las prestaciones en el conjunto de test

MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver="sgd",learning_rate='constant',learning_rate_init= 0.0029224489795918367,hidden_layer_sizes=(29,)) 

MLP_model=MLP.fit(X_train_norm, Y_train)
y_pred=MLP_model.predict(X_test_norm)
print('Accuracy:', accuracy_score(Y_test, y_pred))
print('Sensibility',recall_score(Y_test, y_pred))

- Porcentaje de observaciones consideradas al aplicar la técnica “early stopping”. Opcional

In [ ]:
MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver="sgd",learning_rate='constant',learning_rate_init= 0.0029224489795918367,hidden_layer_sizes=(29,),early_stopping=True,verbose=True)
scores= cross_val_score(MLP, X_train_norm, Y_train, cv=3, scoring='accuracy') 
#print('Score',scores)
#cv_neu_scores.append(scores.mean()) #almacena la media de los scores


Si utilizamos *early stopping* en el clasificador, mediante el parámetro *verbose* se puede observar que en cada fold para en la época 12, ya que la función de pérdida no disminuye en las dos últimas iteraciones. Por lo que considera que está muy cerca del mínimo global y de esta forma no sobreajusta. Sin embargo, se observa que el accuracy obtenido es muy bajo (0.48), por lo que se podría pensar que lo que el *early stopping* considera como un mínimo global en realidad es un mínimo local. Por tanto, sería necesario modificar el valor del parametro *momentum*.

In [ ]:
MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver="sgd",learning_rate='constant',learning_rate_init=0.0029224489795918367,hidden_layer_sizes=(29,), early_stopping=True) 

MLP_model=MLP.fit(X_train_norm, Y_train)
y_pred=MLP_model.predict(X_test_norm)
print('Accuracy:', accuracy_score(Y_test, y_pred))
print('Matriz de confusión:')
print(confusion_matrix(Y_test,y_pred))

Por todo lo explicado anteriormente, se puede observar como el clasificador no detecta correctamente las clases.

- Tamaño del mini-batch. Opcional

Batch es el número de datos que tiene cada iteración de una época, esto es útil porque la red neuronal actualiza los parámetros de los pesos más veces, consiguiendo caracterizar mejor a la población, además de que ayuda computacionalmente a que vaya más rápido. 


Se considera un parámetro libre, con lo que por medio de cross validation se consigue el tamaño óptimo del batch.

In [ ]:
batch = range(10,100)##
cv_batch_scores=[]
for k in batch:
    
    MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver="sgd",learning_rate='constant',learning_rate_init=0.0029224489795918367,hidden_layer_sizes=(29,),batch_size=k )
    scores= cross_val_score(MLP, X_train_norm, Y_train, cv=3, scoring='accuracy') 
    #calcula el accuracy con el clasificador MLP en train y 3 folds
    #print('Score',scores)
    cv_batch_scores.append(scores.mean()) #almacena la media de los scores
    

    
plt.plot(batch, cv_batch_scores)
plt.xlabel('batch')
plt.ylabel('CV accuracy')
plt.show()


print("El número de batch más adecuado es:",np.array(batch)[cv_batch_scores.index(np.array(cv_batch_scores).max())])
print("Accuracy:", max(cv_batch_scores))

In [ ]:
MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver="sgd",learning_rate='constant',learning_rate_init=0.0029224489795918367,hidden_layer_sizes=(29,),batch_size=63) 

MLP_model=MLP.fit(X_train_norm, Y_train)
y_pred=MLP_model.predict(X_test_norm)
print('Accuracy:', accuracy_score(Y_test, y_pred))
print('Sensibility',recall_score(Y_test, y_pred))

- Si hace uso o no (y por qué) de aleatorización de las observaciones durante el aprendizaje. Opcional

Para caracterizar mejor a la población, se hace uso de shuffle para aleatorizar las observaciones en cada época.

In [ ]:
MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver="sgd",learning_rate='constant',learning_rate_init=0.0029224489795918367,hidden_layer_sizes=(29,),batch_size=63, shuffle=False) 

MLP_model=MLP.fit(X_train_norm, Y_train)
y_pred=MLP_model.predict(X_test_norm)
print('Accuracy:', accuracy_score(Y_test, y_pred))
print('Sensibility',recall_score(Y_test, y_pred))

In [ ]:
MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver="sgd",learning_rate='constant',learning_rate_init=0.0029224489795918367,hidden_layer_sizes=(29,),batch_size=63, shuffle=True) 

MLP_model=MLP.fit(X_train_norm, Y_train)
y_pred=MLP_model.predict(X_test_norm)
print('Accuracy:', accuracy_score(Y_test, y_pred))
print('Sensibility',recall_score(Y_test, y_pred))

Por defecto shuffle = True, con lo que se obtienen mejores prestaciones aleatorizando las observaciones en cada época. 

- Número máximo de épocas. Explique de manera breve y concisa qué es una época, ejemplificando con su caso particular.

Una época es el número de veces que se ejecutaran los algoritmos de forwardpropagation y backpropagation. En cada época  todos los datos de entrenamiento pasan por la red neuronal para que esta aprenda sobre ellos. Por ejemplo, en este caso, al usar 5000 ciclos o épocas y 375 datos de entrenamiento, en cada ciclo los 375 datos pasaran por la red neuronal.

Hemos utilizado un *max_iter* de 5000 épocas en el cálculo de cada uno de los parámetros para que converja la función de coste. Sin embargo, con el clasificador ya definido con todos los parámetros óptimos no se necesitan tantas épocas para converger. Esto se puede ver con el parámetro *verbose* a continuación:

In [ ]:
MLP = MLPClassifier(random_state=1, max_iter=5000, activation="logistic", solver="sgd",learning_rate='constant',learning_rate_init=0.0029224489795918367,hidden_layer_sizes=(29,),batch_size=63, verbose = True) 

MLP_model=MLP.fit(X_train_norm, Y_train)
y_pred=MLP_model.predict(X_test_norm)
print('Accuracy:', accuracy_score(Y_test, y_pred))
print('Sensibility',recall_score(Y_test, y_pred))

Como se comentaba arriba, se puede observar con este resultado  que para que la función de coste converja no son necesarias tantas iteraciones ya que se logra con muchas menos, en este caso 307. Esto se ve mediante la función de pérdida ya que llega a un punto en el que no sigue disminuyendo, y ese punto es en la época 307.

In [ ]:
MLP = MLPClassifier(random_state=1, max_iter=307, activation="logistic", solver="sgd",learning_rate='constant',learning_rate_init=0.0029224489795918367,hidden_layer_sizes=(29,),batch_size=63) 

MLP_model=MLP.fit(X_train_norm, Y_train)
y_pred1=MLP_model.predict(X_test_norm)
print('Accuracy:', accuracy_score(Y_test, y_pred1))
print('Sensibility',recall_score(Y_test, y_pred1))
print("Confusión matrix: ")
print(confusion_matrix(Y_test, y_pred1))
print('Specificity:',confusion_matrix(Y_test, y_pred1)[0][0]/(confusion_matrix(Y_test, y_pred1)[0][0]+confusion_matrix(Y_test, y_pred1)[0][1]))
print("Precision:", precision_score(Y_test, y_pred1))

**Realice el aprendizaje de todas las topologías, i.e., considerando al menos n_hidden_1, n_hidden_2 y n_hidden_3 neuronas en la capa oculta. Elija varias medidas de prestaciones (adecuadas para la tarea a resolver) y construya una tabla indicando, para cada topología del MLP, dichas prestaciones sobre el conjunto de validación.**

Para cada nuevo parámetro del clasificador MLP se ha ido acumulando la mejor opción del anterior, excepto para el parámetro *early stopping* que por sus malos resultados, no se ha utilizado en las siguientes. 

In [ ]:
Tabla1 = pd.DataFrame({ "Prestaciones en el conjunto de validacion":["Accuracy"],
                      "activation = logistic" : ["0.731"],
                      "solver = sgd" : ["0.731"],
                       "learning_rate = constant": ["0.731"],
                    "learning_rate_init=0.0029" : ["0.739"],
                     "hidden_layer_sizes=29" : ["0.749"],
                       "early_stopping=True" : ["0.48"]})
Tabla1

El accuracy se mantiene constante en los tres primeros parámetros, esto se debe a que éstos son los default. A partir de *learning rate init* el accuracy va aumentando. Este aumento se debe a que los parámetros han sido elegidos uno a uno, mediante cross validation, y van ajustando mejor el modelo. Lo siguiente que se puede observar es que el parámetro *early stopping* disminuye significativamente el accuracy por lo comentado anteriormente. 

**Justifique razonadamente la elección de una topología y obtenga las prestaciones sobre el conjunto de test.**

In [ ]:
Tabla1 = pd.DataFrame({ "Prestaciones en el conjunto de test":["Accuracy","Sensibility"],
                      "activation = logistic" : ["0.739","0.716"],
                      "solver = sgd" : ["0.739","0.704"],
                       "learning_rate = constant": ["0.739","0.704"],
                    "learning_rate_init=0.0029" : ["0.739","0.716"],
                     "hidden_layer_sizes=29" : ["0.732","0.704"],
                       "early_stopping=True" : ["0.503","-"],
                      "batch = 63": ["0.745", "0.716"]})
Tabla1

Se perciben leves diferencias en el accuracy y la sensibilidad modificando los distintos parámetros elegidos con cross validation. Es cierto, que en validación observamos que el accuracy va aumentando a medida que se van implementando más parámetros. Sin embargo, en test no tienen por que reflejarse las mismas mejoras. Si se realizara otra partición con un conjunto de test diferente, las prestaciones mejorarían o empeorarían. Por tanto, las prestaciones dependen mucho del conjunto de test y por ello, sería aconsejable repetir el proceso y reflejar las prestaciones como la media ± desviación estándar.

## Comparación de resultados

**A la vista de los resultados obtenidos para resolver la tarea de clasificación planteada, justifique brevemente qué modelo considera más adecuado.**

In [3]:
Tabla1 = pd.DataFrame({ "Prestaciones":["Accuracy","Sensibility",'Specificity','Precision'],
                      "KNN" : ["0.733","0.778",'0.637','0.691'],
                      "Árbol de decisión" : ["0.720","0.753",'0.688','0.709'],
                    "MLP" : ["0.745","0.716",'0.775','0.763']})
Tabla1

,Prestaciones,KNN,Árbol de decisión,MLP
0,Accuracy,0.733,0.720,0.745
1,Sensibility,0.778,0.753,0.716
2,Specificity,0.637,0.688,0.775
3,Precision,0.691,0.709,0.763


A una primera vista MLP es el modelo que mejor se ajusta a nuestros datos, dado que presenta un mayor accuracy, el cual es la cantidad de casos detectados correctamente. Además de destacar en especificidad y en precisión, lo cual significa que detecta muy bien a los no diabéticos y que cuando detecta a los diabéticos lo hace con seguridad.
Sin embargo, el clasificador kNN y el árbol de decisión tienen una mayor sensibilidad, es decir, detectan mejor a los casos positivos como realmente enfermos (diabéticos).